<a href="https://colab.research.google.com/github/JimKing100/DS-Unit-2-Regression-Classification/blob/master/module2/assignment_regression_classification_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports

In [0]:
# Imports
%%capture
!pip install --upgrade pandas-profiling plotly
!pip install fiona
!pip install geopandas

In [0]:
# Disable warning
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

### Load Data from GitHub

In [57]:
# Pull data files from GitHub to local directory
import os, sys

# Pull files from Github repo
os.chdir('/content')
!git init .
!git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification.git
!git pull origin master
    
# Change into directory for module
os.chdir('module1')

Reinitialized existing Git repository in /content/.git/
fatal: remote origin already exists.
From https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification
 * branch            master     -> FETCH_HEAD
Already up to date.


In [0]:
# Load the data
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv('../data/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

### Example of Neighborhood Lookup Using Geopandas and Geojson Maps

In [59]:
# Use NYC geojson map to assign neighborhoods based on long/lat coordinates
import geopandas
import json
from shapely.geometry import shape, Point

nyc = geopandas.read_file('https://raw.githubusercontent.com/JimKing100/DS-Unit-2-Regression-Classification/master/module2/NYC.geojson')

# The neighbohood lookup function
def neighborhood(long, lat):
  # construct point based on long/lat returned by geocoder
  point = Point(long, lat)
  n_name = 'Unknown'

# check each polygon to see if it contains the point
  for i in range(0, len(nyc.index)):
    polygon = shape(nyc.loc[i]['geometry'])
    if polygon.contains(point):
      n_name = nyc.loc[i]['ntaname']
      return n_name
    
  return  n_name


longitude = -73.9667
latitude = 40.7947

n_name = neighborhood(longitude, latitude)
print(n_name)

Upper West Side


### Split Data in Train and Test Based on Date

In [0]:
# Convert created feature to datetime
df['created'] = pd.to_datetime(df['created'], infer_datetime_format=True)

# Split off the train data by April and May
mask = (df['created'] > '2016-03-31') & (df['created'] < '2016-06-01')
train = df.loc[mask]

# Split of the test data by June
mask = (df['created'] >= '2016-06-01')
test = df.loc[mask]

### Build Train Features - no_amenities, pets, rooms, interest, desc_len

In [61]:
# Create an amenitites dictionary and sum the values in a new feature no_amenities
amenities = ['elevator',	'cats_allowed',	'hardwood_floors',	'dogs_allowed',
              'doorman',	'dishwasher',	'no_fee',	'laundry_in_building',	'fitness_center',
              'pre-war',	'laundry_in_unit',	'roof_deck',	'outdoor_space',	'dining_room',
              'high_speed_internet',	'balcony',	'swimming_pool',	'new_construction',
              'terrace',	'exclusive',	'loft',	'garden_patio',	'wheelchair_access',	'common_outdoor_space']
train['no_amenities'] = train[amenities].sum(axis=1)

# Create a pets dictionary and sum the values in a new feature no_amenities
pets = ['cats_allowed', 'dogs_allowed']
train['pets'] = train[pets].sum(axis=1)

# Create a rooms dictionary and sum the values in a new feature no_amenities
rooms = ['bedrooms', 'bathrooms']
train['rooms'] = train[rooms].sum(axis=1)

# Create an interest level dictionary and map to values
interest = {'low': 1,'medium:': 2,'high': 3}
train['interest'] = train['interest_level'].map(interest)
values = {'interest': 0}
train = train.fillna(value=values)

# Create a decription length feature
train['desc_len']  = train['description'].str.len()
values = {'desc_len': 0}
train = train.fillna(value=values);

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata

### Build Train Features - Create Neighborhood Feature Using Maps

In [0]:
# Create a long/lat dataframe called hoods
hoods = train[['longitude', 'latitude']]

# This code was run, but takes a lot of time, so results were saved to a file
# The code adds the neighbohood name to the hoods dataframe

#hoods['neighborhood'] = hoods.apply(lambda x: neighborhood(x['longitude'], x['latitude']), axis=1)
#hoods.head()

# This reads in the file output from the previous line of code
n_hoods = pd.read_csv('https://raw.githubusercontent.com/JimKing100/DS-Unit-2-Regression-Classification/master/module2/hoods.csv')
n_hoods = n_hoods.rename(columns={'Unnamed: 0': 'old_index'})

# The train data is merged with the hoods data providing the neighbohood name as a new feature
new_train = pd.merge(train, n_hoods, how='left', left_index=True, right_on=['old_index'])

# A neighborhood dictionary is created to convert names to numbers
neighborhood_dict = {
 'Midwood': 1,
 'Bedford:': 2,
 'Fordham South': 3,
 'Borough Park': 4,
 'Rugby-Remsen Village': 5,
 'East Flushing': 6,
 'Woodhaven': 7,
 'Madison': 8,
 'Auburndale': 9,
 'Williamsbridge-Olinville': 10,
 'Murray Hill': 11,
 'East Elmhurst': 12,
 'Brownsville': 13,
 'East New York (Pennsylvania Ave)':14,
 'Kensington-Ocean Parkway': 15,
 'Parkchester': 16,
 'Erasmus': 17,
 'Cambria Heights': 18,
 'East Flatbush-Farragut': 19,
 'Ocean Parkway South': 20,
 'Starrett City': 21,
 'Morrisania-Melrose': 22,
 'Elmhurst': 23,
 'East Village': 24,
 'Glen Oaks-Floral Park-New Hyde Park': 25,
 'Longwood': 26,
 'Yorkville': 27,
 'Upper East Side-Carnegie Hill': 28,
 'Windsor Terrace': 29,
 'Hammels-Arverne-Edgemere': 30,
 'Rikers Island': 31,
 'Hunts Point': 31,
 'Jackson Heights': 32,
 'Bath Beach': 33,
 'Old Town-Dongan Hills-South Beach': 34,
 'Flatbush': 35,
 'Melrose South-Mott Haven North': 36,
 'Ocean Hill': 37,
 'Morningside Heights': 38,
 'Soundview-Bruckner': 39,
 'Allerton-Pelham Gardens': 40,
 'Jamaica Estates-Holliswood': 41,
 'Hollis': 42,
 'Flatlands': 43,
 'East New York': 44,
 'Kingsbridge Heights': 45,
 'Springfield Gardens North': 46,
 'Canarsie': 47,
 'Norwood': 48,
 'Manhattanville': 49,
 'West Village': 50,
 'Grasmere-Arrochar-Ft. Wadsworth': 51,
 'Queens Village': 52,
 'Chinatown': 53,
 'Pelham Bay-Country Club-City Island': 54,
 'Woodlawn-Wakefield': 55,
 'Old Astoria': 56,
 'Astoria': 57,
 'Stuyvesant Town-Cooper Village': 58,
 'Dyker Heights': 59,
 'Bensonhurst West': 60,
 'West New Brighton-New Brighton-St. George': 61,
 'New Brighton-Silver Lake': 62,
 'Westerleigh': 63,
 'University Heights-Morris Heights': 64,
 'Bayside-Bayside Hills': 65,
 'Crown Heights North': 66,
 'East Concourse-Concourse Village': 67,
 'North Corona': 68,
 'Cypress Hills-City Line': 69,
 'Kew Gardens Hills': 70,
 'Pomonok-Flushing Heights-Hillcrest': 71,
 'North Side-South Side': 72,
 'Lower East Side': 73,
 'Greenpoint': 74,
 'Spuyten Duyvil-Kingsbridge': 75,
 'Sunset Park East': 76,
 'Marble Hill-Inwood': 77,
 'Homecrest': 78,
 'Washington Heights North': 79,
 'Steinway': 80,
 'Mott Haven-Port Morris': 81,
 'West Brighton': 82,
 'Central Harlem North-Polo Grounds': 83,
 'Queensbridge-Ravenswood-Long Island City': 84,
 'New Dorp-Midland Beach': 85,
 'Van Cortlandt Village': 86,
 'Co-op City': 87,
 'Bay Ridge': 88,
 'Sunset Park West': 89,
 'Fort Greene': 90,
 'SoHo-TriBeCa-Civic Center-Little Italy': 91,
 'Battery Park City-Lower Manhattan': 92,
 'Clinton': 93,
 'Prospect Heights': 94,
 'Baisley Park': 95,
 'South Jamaica': 96,
 'Ozone Park': 97,
 'Georgetown-Marine Park-Bergen Beach-Mill Basin': 98,
 'Brighton Beach': 99,
 'Bensonhurst East': 100,
 'West Farms-Bronx River': 101,
 'Sheepshead Bay-Gerritsen Beach-Manhattan Beach': 102,
 'Westchester-Unionport': 103,
 'Oakwood-Oakwood Beach': 104,
 'Grymes Hill-Clifton-Fox Hills': 105,
 'Park Slope-Gowanus': 106,
 'Stapleton-Rosebank': 107,
 'Fresh Meadows-Utopia': 108,
 'Bellerose': 109,
 'Stuyvesant Heights': 110,
 'East Williamsburg': 111,
 'Ridgewood': 112,
 'East Harlem South': 113,
 'Rego Park': 114,
 'East Harlem North': 115,
 'Bushwick North': 116,
 'Bushwick South': 117,
 'Central Harlem South': 118,
 'College Point': 119,
 'Midtown-Midtown South': 120,
 'Glendale': 121,
 'Charleston-Richmond Valley-Tottenville': 122,
 'park-cemetery-etc-Staten Island': 123,
 'New Springville-Bloomfield-Travis': 124,
 'Todt Hill-Emerson Hill-Heartland Village-Lighthouse Hill': 125,
 'South Ozone Park': 126,
 'Lindenwood-Howard Beach': 127,
 'Prospect Lefferts Gardens-Wingate': 128,
 'Murray Hill-Kips Bay': 129,
 "Mariner's Harbor-Arlington-Port Ivory-Graniteville": 130,
 'Crown Heights South': 131,
 'Brooklyn Heights-Cobble Hill': 132,
 'Port Richmond': 133,
 'Hunters Point-Sunnyside-West Maspeth': 134,
 'Claremont-Bathgate': 135,
 'Van Nest-Morris Park-Westchester Square': 136,
 'Pelham Parkway': 137,
 'Mount Hope': 138,
 'Ft. Totten-Bay Terrace-Clearview': 139,
 'Whitestone': 140,
 'Turtle Bay-East Midtown': 141,
 'Lenox Hill-Roosevelt Island': 142,
 'Elmhurst-Maspeth': 143,
 'Woodside': 144,
 'St. Albans': 145,
 'Laurelton': 146,
 'Hamilton Heights': 147,
 'Jamaica': 148,
 'Richmond Hill': 149,
 'Briarwood-Jamaica Hills': 150,
 'Kew Gardens': 151,
 'Middle Village': 152,
 'Maspeth': 153,
 'Upper West Side': 154,
 'Lincoln Square': 155,
 'Bronxdale': 156,
 'Oakland Gardens': 157,
 'Douglas Manor-Douglaston-Little Neck': 158,
 "Annadale-Huguenot-Prince's Bay-Eltingville": 159,
 'Great Kills': 160,
 'Seagate-Coney Island': 161,
 'Corona': 162,
 'Schuylerville-Throgs Neck-Edgewater Park': 163,
 'Gravesend': 164,
 'East Tremont': 165,
 'North Riverdale-Fieldston-Riverdale': 166,
 'Bedford Park-Fordham North': 167,
 'Belmont': 168,
 'Eastchester-Edenwald-Baychester': 169,
 'Breezy Point-Belle Harbor-Rockaway Park-Broad Channel': 170,
 'Crotona Park East': 171,
 'Rossville-Woodrow':172,
 'Arden Heights': 173,
 'Far Rockaway-Bayswater': 174,
 'Soundview-Castle Hill-Clason Point-Harding Park': 175,
 'park-cemetery-etc-Bronx': 176,
 'Carroll Gardens-Columbia Street-Red Hook': 177,
 'park-cemetery-etc-Manhattan': 178,
 'Rosedale': 179,
 'Flushing': 180,
 'Queensboro Hill': 181,
 'Hudson Yards-Chelsea-Flatiron-Union Square': 182,
 'Gramercy': 183,
 'DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill': 184,
 'Clinton Hill': 185,
 'Williamsburg': 186,
 'park-cemetery-etc-Brooklyn': 187,
 'Washington Heights South': 188,
 'Highbridge': 189,
 'West Concourse': 190,
 'Forest Hills': 191,
 'park-cemetery-etc-Queens': 192,
 'Springfield Gardens South-Brookville': 193,
 'Airport': 194
}

# The neighborhood names are converted to numbers in feature nid
new_train['nid'] = new_train['neighborhood'].map(neighborhood_dict)

# Null values are filled
values = {'nid': 0}
train = new_train.fillna(value=values)

### Build Train Features - Create Neighborhood Mean Price Feature

In [63]:
# From scipy.spatial.distance import euclidean as get_euclidean_distance
from geopy.distance import vincenty as get_geodesic_distance

# Null values are filled in longitude and latitude
values = {'longitude': 50, 'latitude': -70}
train = train.fillna(value=values)

# Create a long/lat dataframe called nhoods
nhoods = train[['old_index', 'bedrooms', 'longitude_x', 'latitude_x']]

# Function to calculate the mean of the 5 closest n bedroom rentals
def neighbor_mean(beds, source_longitude, source_latitude):

  source_lonlat = source_longitude, source_latitude
  source_table = train[train['bedrooms'] == beds]

  target_table = pd.DataFrame(source_table, columns = ['longitude_x', 'latitude_x', 'price']) 

  def get_distance(row):
      target_lonlat = row['longitude_x'], row['latitude_x']
      return get_geodesic_distance(target_lonlat, source_lonlat).meters

  target_table['distance'] = target_table.apply(get_distance, axis=1)

  # Get the nearest 5 locations
  nearest_target_table = target_table.sort_values(['distance'])[:5]

  return nearest_target_table['price'].mean()

# This code was run, but takes a lot of time, so results were saved to a file
# nhoods['mean_neighbor_price'] = nhoods.apply(lambda x: neighbor_mean(x['bedrooms'], x['longitude_x'], x['latitude_x']), axis=1)

# The train data is merged with the hoods data providing the neighbohood name as a new feature
nhoods = pd.read_csv('https://raw.githubusercontent.com/JimKing100/DS-Unit-2-Regression-Classification/master/module2/nhoods.csv')
new_train = pd.merge(train, nhoods, on=['old_index'])
train = new_train.rename(columns={'bedrooms_x': 'bedrooms'})
print(train.shape)
train.head()

(31844, 49)


,bathrooms,bedrooms,created,description,display_address,latitude_x_x,longitude_x_x,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,no_amenities,pets,rooms,interest,desc_len,old_index,longitude_y,latitude_y,neighborhood,nid,Unnamed: 0,bedrooms_y,longitude_x_y,latitude_x_y,mean_neighbor_price
0,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,2.0,3.0,691.0,2,-74.0018,40.7388,West Village,50.0,0,1,-74.0018,40.7388,3090.0
1,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,2.0,1.0,492.0,3,-73.9677,40.7539,Turtle Bay-East Midtown,141.0,1,1,-73.9677,40.7539,3610.0
2,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,5.0,1.0,479.0,4,-73.9493,40.8241,Hamilton Heights,147.0,2,4,-73.9493,40.8241,3200.0
3,2.0,4,2016-04-19 04:24:47,,West 18th Street,40.7429,-74.0028,7995,350 West 18th Street,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6.0,0.0,8.0,5,-74.0028,40.7429,Hudson Yards-Chelsea-Flatiron-Union Square,182.0,3,4,-74.0028,40.7429,7958.0
4,1.0,2,2016-04-27 03:19:56,Stunning unit with a great location and lots o...,West 107th Street,40.8012,-73.9660,3600,210 West 107th Street,low,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,2,3.0,1.0,579.0,6,-73.9660,40.8012,Morningside Heights,38.0,4,2,-73.9660,40.8012,3600.0


### Build Test Features - no_amenities, pets, rooms, interest, desc_len

In [64]:
test['no_amenities'] = test[amenities].sum(axis=1)
test['pets'] = test[pets].sum(axis=1)
test['rooms'] = test[rooms].sum(axis=1)

test['interest'] = test['interest_level'].map(interest)
values = {'interest': 0}
test = test.fillna(value=values)

test['desc_len']  = test['description'].str.len()
values = {'desc_len': 0}
test = test.fillna(value=values);

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

### Build Test Features - Create Neighborhood Feature Using Maps

In [0]:
tn_hoods = pd.read_csv('https://raw.githubusercontent.com/JimKing100/DS-Unit-2-Regression-Classification/master/module2/thoods.csv')
tn_hoods = tn_hoods.rename(columns={'Unnamed: 0': 'old_index'})

new_test = pd.merge(test, tn_hoods, how='left', left_index=True, right_on=['old_index'])

new_test['nid'] = new_test['neighborhood'].map(neighborhood_dict)
values = {'nid': 0}
test = new_test.fillna(value=values)


### Build Test Features - Create Neighborhood Mean Price Feature

In [0]:
values = {'longitude': 50, 'latitude': -70}
test = test.fillna(value=values)

In [67]:
from geopy.distance import vincenty as get_geodesic_distance
# from scipy.spatial.distance import euclidean as get_euclidean_distance

# Function to calculate the mean of the 5 closest n bedroom rentals
def neighbor_mean(beds, source_longitude, source_latitude):

  source_lonlat = source_longitude, source_latitude
  source_table = test[test['bedrooms'] == beds]

  target_table = pd.DataFrame(source_table, columns = ['longitude_x', 'latitude_x', 'price']) 

  def get_distance(row):
      target_lonlat = row['longitude_x'], row['latitude_x']
      return get_geodesic_distance(target_lonlat, source_lonlat).meters

  target_table['distance'] = target_table.apply(get_distance, axis=1)

  # Get the nearest 5 locations
  nearest_target_table = target_table.sort_values(['distance'])[:5]

  return nearest_target_table['price'].mean()

# This code was run, but takes a lot of time, so results were saved to a file
# tnhoods['mean_neighbor_price'] = tnhoods.apply(lambda x: neighbor_mean(x['bedrooms'], x['longitude_x'], x['latitude_x']), axis=1)  

# The test data is merged with the hoods data providing the neighbohood name as a new feature
tnhoods = pd.read_csv('https://raw.githubusercontent.com/JimKing100/DS-Unit-2-Regression-Classification/master/module2/tnhoods.csv')
new_test = pd.merge(test, tnhoods, on=['old_index'])
test = new_test.rename(columns={'bedrooms_x': 'bedrooms'})

print(test.shape)
test.head()

(16973, 49)


,bathrooms,bedrooms,created,description,display_address,latitude_x_x,longitude_x_x,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,no_amenities,pets,rooms,interest,desc_len,old_index,longitude_y,latitude_y,neighborhood,nid,Unnamed: 0,bedrooms_y,longitude_x_y,latitude_x_y,mean_neighbor_price
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.5,0.0,588.0,0,-73.9425,40.7145,East Williamsburg,111.0,0,3,-73.9425,40.7145,3460.0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,2,3.0,1.0,8.0,1,-73.9667,40.7947,Upper West Side,154.0,1,2,-73.9667,40.7947,5696.0
2,1.0,1,2016-06-03 03:21:22,Check out this one bedroom apartment in a grea...,W. 173rd Street,40.8448,-73.9396,1675,644 W. 173rd Street,low,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,1.0,690.0,11,-73.9396,40.8448,Washington Heights South,188.0,2,1,-73.9396,40.8448,1662.0
3,1.0,1,2016-06-01 03:11:01,Spacious 1-Bedroom to fit King-sized bed comfo...,East 56th St..,40.7584,-73.9648,3050,315 East 56th St..,low,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,2.0,1.0,569.0,14,-73.9648,40.7584,Turtle Bay-East Midtown,141.0,3,1,-73.9648,40.7584,3010.0
4,2.0,4,2016-06-07 04:39:56,SPRAWLING 2 BEDROOM FOUND! ENJOY THE LUXURY OF...,W 18 St.,40.7391,-73.9936,7400,30 W 18 St.,medium,1,1,1,1,1,1,0,0,1,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,11,2,6.0,0.0,870.0,24,-73.9936,40.7391,Hudson Yards-Chelsea-Flatiron-Union Square,182.0,4,4,-73.9936,40.7391,7490.0


### Run Linear Regression Model

In [69]:
# 1. Import the appropriate estimator class from Scikit-Learn
from sklearn.linear_model import LinearRegression
import numpy as np

# 2. Instantiate this class
model = LinearRegression()

# 3. Arrange X features matrix & y target vector
features = ['bedrooms', 'bathrooms', 'elevator',	'cats_allowed',	'hardwood_floors',	'dogs_allowed',
            'doorman',	'dishwasher',	'no_fee',	'laundry_in_building',	'fitness_center',
            'pre-war',	'laundry_in_unit',	'roof_deck',	'outdoor_space',	'dining_room',
            'high_speed_internet',	'balcony',	'swimming_pool',	'new_construction',
            'terrace',	'exclusive',	'loft',	'garden_patio',	'wheelchair_access',	'common_outdoor_space',
            'no_amenities', 'nid', 'pets', 'rooms', 'interest', 'desc_len', 'mean_neighbor_price']
target = 'price'

X_train = train[features]
y_train = train[target]

X_test = test[features]
y_test = test[target]

# 4. Fit the model
model.fit(X_train, y_train)

# 5. Apply the model
y_pred = model.predict(X_train)

# Show the coefficient
print('Coefficients', model.coef_, '\n')

# Show the intercept
print('Intercept', model.intercept_, '\n')

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Print regression metrics
train_mse = mean_squared_error(y_train, y_pred)
train_rmse = np.sqrt(train_mse)
train_mae = mean_absolute_error(y_train, y_pred)
train_r2 = r2_score(y_train, y_pred)
print('Train Mean Squared Error:', train_mse)
print('Train Root Mean Squared Error:', train_rmse)
print('Train Mean Absolute Error:', train_mae)
print('Train R^2:', train_r2)
print('\n')

ty_pred = model.predict(X_test)

# Print regression metrics
test_mse = mean_squared_error(y_test, ty_pred)
test_rmse = np.sqrt(test_mse)
test_mae = mean_absolute_error(y_test, ty_pred)
test_r2 = r2_score(y_test, ty_pred)
print('Test Mean Squared Error:', test_mse)
print('Test Root Mean Squared Error:', test_rmse)
print('Test Mean Absolute Error:', test_mae)
print('Test R^2:', test_r2)



Coefficients [-2.06577742e+02  3.87047675e+02  2.21621783e+01 -2.01895303e+01
 -9.53297082e+01  3.70527785e+01  1.25799528e+02  3.31113715e+00
 -7.71867564e+01 -9.89920257e+01  2.29977618e+01 -2.65589087e+01
  1.13457156e+02 -4.24823520e+01 -4.81432613e+01 -3.87014260e+01
 -1.15253053e+02 -4.28995730e+00 -4.67823581e+01 -5.62436820e+01
  1.10339609e+02  1.14267467e+02  1.41289698e+01  9.42571357e-01
  1.02856442e+02  3.23577896e+00  3.98359096e-01 -1.17279475e-01
  1.68632482e+01  1.80469932e+02 -6.26608394e+00  6.36449975e-02
  8.96511082e-01] 

Intercept -305.29870180042144 

Train Mean Squared Error: 419644.8193362777
Train Root Mean Squared Error: 647.7999840508471
Train Mean Absolute Error: 364.2594287542734
Train R^2: 0.8648499176534774


Test Mean Squared Error: 521064.99636137823
Test Root Mean Squared Error: 721.8483194975092
Test Mean Absolute Error: 406.88765923272365
Test R^2: 0.8323483169980366
